# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [11]:
# import numpy and pandas

import numpy as np
import pandas as pd
import scipy.stats as stats

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [87]:
# pd.read_csv

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# df.head()

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [7]:
# Filtered data

legend = pokemon[pokemon["Legendary"] == True]
regular = pokemon[pokemon["Legendary"] == False]

#Print lengths

print(f'There are {len(legend)} Legendary pokemon')
print(f'There are {len(regular)} regular pokemon')

There are 65 Legendary pokemon
There are 735 regular pokemon


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [10]:
# Print df.mean() + df.std()

print(f'Legendary pokemon have a mean of total points of {round(legend["Total"].mean(),2)} with a std of {round(legend["Total"].std(),2)}')
print(f'Regular pokemon have a mean of total points of {round(regular["Total"].mean(),2)} with a std of {round(regular["Total"].std(),2)}')

Legendary pokemon have a mean of total points of 637.38 with a std of 60.94
Regular pokemon have a mean of total points of 417.21 with a std of 106.76


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [15]:
# ttest_ind + 2 different samples + Welch correction
# p value is smaller than 0.05 - we reject the null hypothesis that two sample have a similar distribution

stat, pval = stats.ttest_ind(legend['Total'], regular['Total'], equal_var=False)
pval

9.357954335957446e-47

What do you conclude from this test? Write your conclusions below.

In [6]:
# We conclude that any of the two different samples will probably have a greater mean, or a wider distribution
# As we checked previously, Legendary pokemon have a higher Totla mean and lower Total std

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [24]:
# df.groupby + count()

pokemon.groupby('Type 1')[['Name']].count().sort_values('Name', ascending=False)

,Name
Type 1,
Water,112
Normal,98
Grass,70
Bug,69
Psychic,57
Fire,52
Electric,44
Rock,44
Ghost,32


Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [25]:
# Filter and mean() and std()

water = pokemon[pokemon["Type 1"] == 'Water']
not_water = pokemon[pokemon["Type 1"] != 'Water']

print(f'Legendary pokemon have a mean of total points of {round(water["Total"].mean(),2)} with a std of {round(water["Total"].std(),2)}')
print(f'Regular pokemon have a mean of total points of {round(not_water["Total"].mean(),2)} with a std of {round(not_water["Total"].std(),2)}')

Legendary pokemon have a mean of total points of 430.46 with a std of 113.19
Regular pokemon have a mean of total points of 435.86 with a std of 121.09


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [26]:
# ttest_ind + 2 different samples + Welch correction
# p value is gerater than 0.05 - we do not reject the null hypothesis that two sample have a similar distribution

stat, pval = stats.ttest_ind(water['Total'], not_water['Total'], equal_var=True)
pval

0.6587140317488793

Write your conclusion below.

In [10]:
# We can conclude that there are not significant differences between water and the rest of types if we consider them as a group

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [27]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [28]:
# Define 2 samples

attack = pokemon['Attack']
defense = pokemon['Defense']

# ttest_rel + 2 related samples
# p value is smaller than 0.05 - we reject the null hypothesis that two sample have a similar distribution

stat, pval = stats.ttest_rel(attack, defense)
pval

1.7140303479358558e-05

In [30]:
# Print df.mean() + df.std()

print(f'Legendary pokemon have a mean of total points of {round(attack.mean(),2)} with a std of {round(attack.std(),2)}')
print(f'Regular pokemon have a mean of total points of {round(defense.mean(),2)} with a std of {round(defense.std(),2)}')

Legendary pokemon have a mean of total points of 79.0 with a std of 32.46
Regular pokemon have a mean of total points of 73.84 with a std of 31.18


Describe the results of the test in the cell below.

In [12]:
# We can conclude that attack and defense score do not have the same distribution. As we have just checked, attack has a higher average score, and they have similar std

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [32]:
sp_attack = pokemon['Sp. Atk']
sp_defense = pokemon['Sp. Def']

# ttest_rel + 2 related samples
# p value is greater than 0.05 - we do not reject the null hypothesis that two sample have a similar distribution

stat, pval = stats.ttest_rel(sp_attack, sp_defense)
pval

0.3933685997548122

Describe the results of the test in the cell below.

In [33]:
# Print df.mean() + df.std()

print(f'Legendary pokemon have a mean of total points of {round(sp_attack.mean(),2)} with a std of {round(sp_attack.std(),2)}')
print(f'Regular pokemon have a mean of total points of {round(sp_defense.mean(),2)} with a std of {round(sp_defense.std(),2)}')

Legendary pokemon have a mean of total points of 72.82 with a std of 32.72
Regular pokemon have a mean of total points of 71.9 with a std of 27.83


In [14]:
# We can conclude that special attack and special defense score have a similar distribution. As we have just checked, they have similar average score, and they have similar std

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [46]:
# Define 2 samples

attack = pokemon['Attack']
defense = pokemon['Defense']

# ttest_1samp by subtracting both means and indicating 0 as popmean

stat, pval = stats.ttest_1samp(attack - defense, 0)
stat, pval2 = stats.ttest_rel(attack, defense)

# Check if ttest_1samp and ttest_rel have same output

pval == pval2

True

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [90]:
# Create new column with Water vs Not Water

pokemon['Water'] = pokemon['Type 1'].apply(lambda x: 'Water' if x == 'Water' else 'Not Water')

# pd.crosstab

crosstab = pd.crosstab(index= pokemon['Legendary'], columns = pokemon['Water'], values=pokemon['Total'], aggfunc='count')
crosstab

Water,Not Water,Water
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [92]:
stats.chi2_contingency(crosstab)

(2.9429200762850503,
 0.08625467249550949,
 1,
 array([[632.1, 102.9],
        [ 55.9,   9.1]]))

Based on a 95% confidence, should we reject the null hypothesis?

In [94]:
# p value is greater than 0.05 - we cannot reject the null hypothesis and therefore we cnanot reject that there is an association between the variables

pval = stats.chi2_contingency(crosstab)[1]
pval

0.08625467249550949